# Big Data 5 - Spark GraphX

In [1]:
%login montana.dataapplab.com

[ssh] Login to montana.dataapplab.com...
[ssh] host=montana.dataapplab.com hostname=montana.dataapplab.com other_conf={'user': 'dcdsdepeizhang', 'port': 49233, 'keyfile': ['/home/dz3vg/.ssh/id_rsa'], 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7f6b7c443a90>}
[ssh] forwarding local agent
[ssh] Successfully logged in.


In [2]:
pwd

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang
/home/dcdsdepeizhang


In [3]:
ls

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang
data
demo


In [4]:
cd demo/

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang
[ssh] new cwd: /home/dcdsdepeizhang/demo


In [5]:
cat << EOF > spark-graphx.py

from pyspark import SparkContext
from pyspark.sql import HiveContext
from graphframes import *

sc = SparkContext()
sc.setLogLevel("ERROR")
SQLContext = HiveContext(sc)

vertices = SQLContext.createDataFrame([ 
    ("a", "Alice",   34), 
    ("b", "Bob",     36), 
    ("c", "Charlie", 30), 
    ("d", "David",   29), 
    ("e", "Esther",  32), 
    ("f", "Fanny",   36), 
    ("g", "Jason",   36), 
    ("h", "Celine",  26) 
], ["id", "name", "age"])

edges = SQLContext.createDataFrame([ 
    ("a", "e", "friend"), 
    ("f", "b", "follow"), 
    ("c", "e", "friend"), 
    ("a", "b", "friend"), 
    ("b", "c", "follow"), 
    ("c", "b", "follow"), 
    ("f", "c", "follow"), 
    ("e", "f", "follow"), 
    ("e", "d", "friend"), 
    ("d", "a", "friend"), 
    ("g", "h", "friend") 
], ["src", "dst", "relationship"])

print "Create Graph\n"
graph = GraphFrame(vertices, edges)
graph.vertices.show()
graph.edges.show()

print "Graph Degree\n"
graph.degrees.show()
graph.outDegrees.show()
graph.inDegrees.show()

print "Find Path\n"
graph.find('(A)-[]->(B); (B)-[]->(C); !(A)-[]->(C); !(C)-[]->(A)').show()
graph.shortestPaths(landmarks=["a"]).show()

print "SubGraph\n"
vertices1 = graph.vertices.filter("age > 30")
edges1 = graph.edges.filter("relationship = 'friend'")
graph1 = GraphFrame(vertices1, edges1)
graph1.vertices.show()
graph1.edges.show()

print "Connected Component\n"
graph.connectedComponents().show()
graph.stronglyConnectedComponents(maxIter=10).show()

print "PageRank\n"
results = graph.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.show()
results.edges.show()

EOF

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo


In [6]:
pyspark --packages graphframes:graphframes:0.1.0-spark1.6 spark-graphx.py

[ssh] host = montana.dataapplab.com, cwd = /home/dcdsdepeizhang/demo
Multiple versions of Spark are installed but SPARK_MAJOR_VERSION is not set
Spark1 will be picked by default
Use ./bin/spark-submit <python file>
Ivy Default Cache set to: /home/dcdsdepeizhang/.ivy2/cache
The jars for the packages stored in: /home/dcdsdepeizhang/.ivy2/jars
:: loading settings :: url = jar:file:/usr/hdp/2.5.3.0-37/spark/lib/spark-assembly-1.6.2.2.5.3.0-37-hadoop2.7.3.2.5.3.0-37.jar!/org/apache/ivy/core/settings/ivysettings.xml
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent;1.0
	confs: [default]
	found graphframes#graphframes;0.1.0-spark1.6 in spark-packages
:: resolution report :: resolve 190ms :: artifacts dl 3ms
	:: modules in use:
	graphframes#graphframes;0.1.0-spark1.6 from spark-packages in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifac

In [7]:
%logout

[ssh] Closing existing connection.
[ssh] Successfully logged out.
